In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import glob
import scipy
import copy

#plotting with Cartopy. 
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import cm
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('font',**{'family':'serif','serif':['Times']})
rc('text', usetex=True)
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# import scipy
from datetime import datetime
import os
import importlib


# import statsmodels.api as sm
from scipy import stats
from shapely.geometry.polygon import LinearRing
from dask.diagnostics import ProgressBar
import xesmf as xe
from datetime import timedelta


In [ ]:
os.getcwd()

In [ ]:
from distributed import Client
from ncar_jobqueue import NCARCluster
cluster = NCARCluster(project='P54048000',walltime='04:30:00')
cluster.scale(10)
client = Client(cluster)
client

In [ ]:
New_path = '/glade/scratch/wchapman/inputdata/nudging/ERA5//' 
FNS_to_regrid = sorted(glob.glob('/glade/scratch/juliob/ERA5_regridded/*.nc'))

for ee,fnfn in enumerate(FNS_to_regrid):
    print('doing: ',fnfn)
    
    print('!!!!CHANGING TO PA ... THIS COULD NEED TO BE CHANGED!!!!!!')
    NU_ds = xr.open_dataset(fnfn)
    NU_ds = NU_ds.assign_coords(lev=NU_ds.lev/100)
    print('!!!!CHANGING TO PA ... THIS COULD NEED TO BE CHANGED!!!!!!')
    #need a template file
    Nudge_Path=sorted(glob.glob('/glade/scratch/wchapman/inputdata/nudging/ERAI_fv09/L32/*.nc'))
    DSe = xr.open_dataset(Nudge_Path[0],decode_times=False)
    
    newlat = DSe['slat']
    newlon = DSe['lon']
    ds_out = xr.Dataset({'lat': newlat, 'lon': newlon})
    #need a template file
    
    print('... beginning to regrid US ! ...')
    regridder = xe.Regridder(NU_ds['U'], ds_out, 'bilinear')
    BeanTown = regridder(NU_ds.U)
    print('done regrid')
    
    NU_Us = xr.Dataset(
        {
            "US":(["lev","slat","lon"],BeanTown.data), 
        },
        coords={
            "lev":np.array(DSe.lev),
            "slat":np.array(DSe.slat),
            "lon":np.array(DSe.lon),
        },)
    
    
    print('... beginning to regrid VS ! ...')
    NV_ds = xr.open_dataset(fnfn)
    NV_ds = NU_ds.assign_coords(lev=NV_ds.lev/100)
    ##need a template file
    Nudge_Path=sorted(glob.glob('/glade/scratch/wchapman/inputdata/nudging/ERAI_fv09/L32/*.nc'))
    DSe = xr.open_dataset(Nudge_Path[0],decode_times=False)
    
    newlat = DSe['lat']
    newlon = DSe['slon']
    ds_out = xr.Dataset({'lat': newlat, 'lon': newlon})
    regridder = xe.Regridder(NV_ds, ds_out, 'bilinear')
    BeanTown = regridder(NV_ds.V)
    ##need a template file
    
    NV_Vs = xr.Dataset(
        {
            "VS":(["lev","lat","slon"],BeanTown.data), 
        },
        coords={
            "lev":np.array(DSe.lev),
            "lat":np.array(DSe.lat),
            "slon":np.array(DSe.slon),
        },)

    NU_out= xr.merge([NU_ds,NU_Us,NV_Vs])   
    for ii in NU_out.data_vars:
        if ii in DSe:
            NU_out[ii].attrs = DSe[ii].attrs
    
    NU_out = NU_out.assign_coords(time=pd.to_datetime(fnfn[-19:-9]+' '+ str(timedelta(seconds=float(fnfn[-8:-3]))), format='%Y-%m-%d %H:%M:%S'))
    expand_time = ['U','US','V','VS','T','Q','PS','PHIS','W','PHIS_ERA']
    for var_in in expand_time:
        NU_out[var_in]=NU_out[var_in].expand_dims('time')
    
    print('...saving out...')
    #save it all     
    svname = New_path + 'TESTING_UsVs_'+fnfn.split('/')[-1]
    print(svname)
    NU_out.to_netcdf(svname,format="NETCDF3_CLASSIC")
    print("##########################")